In [9]:
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from numpy.f2py.auxfuncs import throw_error

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings



### Création de l'authentification API Spotify

In [40]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

SPOTIFY_CLIENT_ID = "ed9ece4a286d42f1aa834c82bed1f95e"
SPOTIFY_CLIENT_SECRET = "e1f250bb36914f3c9235c88df325eff6"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri="http://localhost:8000/callback",
    scope="user-top-read playlist-read-private user-library-read playlist-modify-public playlist-modify-private",
    cache_path="./spotify_cache",
    state="unique_state_value"
))

In [39]:
import os

if os.path.exists("./spotify_cache"):
    os.remove("./spotify_cache")
    print("Custom cache file deleted.")
else:
    print("Custom cache file not found.")

Custom cache file deleted.


### Get the user's top tracks

In [41]:
top_tracks = sp.current_user_top_tracks(limit=50, time_range='long_term')

print("Your top tracks:")

for idx, track in enumerate(top_tracks['items']):
    print(f"{idx + 1}: {track['name']} by {track['artists'][0]['name']}")
    

Your top tracks:
1: 669 Tah by Mini RTTCLAN
2: Kpop by Freeze corleone
3: La Quête by Orelsan
4: Double K by Kaaris
5: Dimanche by Leo Roi
6: Ancelotti by Freeze corleone
7: M-A-espace-M-I-E by MPL
8: Washing Machine Heart by Mitski
9: Orézon by Oré
10: Apocalypse by Kaaris
11: Le roi des ombres - Extrait de la BO 'Le roi des ombres' by Kaaris
12: Se-vrak by Kaaris
13: Jeune intermittent by Spider ZED
14: Crève-cœur by Adèle Castillon
15: 7 sur 7 by Koba LaD
16: Blue by Kerchak
17: Eurovision (feat. Rondodasosa, Baby Gang, A2 Anti, Morad, Beny Jr, Ashe 22 & Freeze corleone) by Central Cee
18: Règne sur le monde by Freeze corleone
19: CP_009_ Évidemment by Orelsan
20: Dernière danse by Kyo
21: Violette & citronnelle by ZZCCMXTP
22: Ta reine by Angèle
23: Diarabi by Kaaris
24: Adieu les filles by KronoMuzik
25: My Love Mine All Mine by Mitski
26: FRAUDE by Amine Farsi
27: Metaverse by Rim'K
28: MW2 by Freeze corleone
29: Cesse cesse by Clou
30: Zoo by Kaaris
31: Oui, je l'adore by Paulin

### Get the track IDs to fetch audio features

In [42]:
track_ids = [track['id'] for track in top_tracks['items']]

print(f"Track IDs: {track_ids}")

Track IDs: ['4sEq16gV91gLn4xgfbDPQt', '6JTFR9jAJ9z8jUiGo2q1PV', '0w6lsLBvhtGcqMYA7MB7r6', '5NV2E7EClr01hmQYLyoDOr', '2ap3iiAG2VJ027m3HtleZL', '3jBIofMaOimfUgK4E3hBGB', '6gpfhniHLBtdj5E1a1VM1S', '3jjsRKEsF42ccXf8kWR3nu', '27gxyjy1stzbiaYUJP719r', '479TjlDwB0JAyPjZjxFah8', '3RXT1XhZSOgkD2pjzXBLHj', '2hUn92NSiQc3MBKYjhgUgc', '2Yy5cJk1eDZ7rRAND119W1', '17Kr5yFiPWlfWJo3c65sE1', '09MdPU11JSdnJGfAGlHHf3', '7wBoSW48q4ZFe8qSdozqqi', '4f5TKGvkD4TkTjr61GKNAG', '363mqittn6QH7w5qMmsEB0', '6btgTbK2UslfSu0qjTEXQm', '3UYc8Rc0hskiAzyvczieGl', '2INs1g6Vfcxo3BWxfoG51U', '3iXhLi33EpUfcOwasC8un1', '1BuOKZA8fokaiEVqBkIHF1', '4RPBN897qKd3wEsxmtSUqd', '3vkCueOmm7xQDoJ17W1Pm3', '5KuU8cuhCGJRmAUZsqUBRE', '7rzjdyHmqwloCA16UBdDgg', '2sMxeHjS9rQh9gB92rcRF9', '1i5nz3R4l37hQ2NmccbXxZ', '3BaGyruhFKKP6f8e2koRri', '6w2Kgprot9BesN76ka2NwK', '2J0AJV5DsKyDuX1qEgJngB', '5cTitJ7nqx8eeJCNIgdHjp', '6DPrYPPGYK218iVIZDix3i', '4Aam8WAOsj5CwZwYcjSCWD', '6mlYa6FsNSfpBtr8axmVoc', '30d0q6kt1BIfwAQUCAfxVQ', '6cryNSlBitzoR2Y5HROrii', 

### Load data_appended dataset and fetch the top tracks using ids

In [43]:
data = pd.read_csv("data/data_appended.csv")

top_tracks_data = data[data['id'].isin(track_ids)]

print("Top tracks data :")
print("Found a total of {} tracks".format(len(top_tracks_data)))
print(top_tracks_data)

track_to_analyse = top_tracks_data['id'].tolist()

print(track_to_analyse)
print("Tracks that are not in top track :")
for idx,track in enumerate(top_tracks['items']):
    if track['id'] not in top_tracks_data['id'].values:
        print(f"{idx}: {track['name']} by {track['artists'][0]['name']}")

Top tracks data :
Found a total of 3 tracks
        valence  year  acousticness            artists  danceability  \
19263     0.483  2018        0.7400  ['Alec Benjamin']         0.652   
200896    0.582     0        0.1330          ['Rilès']         0.670   
215676    0.567     0        0.0092         ['Mitski']         0.613   

        duration_ms  energy  explicit                      id  \
19263      169354.0   0.557         0  2qxmye6gAegTMjLKEBoR3d   
200896     182044.0   0.630         1  45KqDHTOJtIM53QR1gPGTa   
215676     128198.0   0.822         0  3jjsRKEsF42ccXf8kWR3nu   

        instrumentalness  key  liveness  loudness  mode  \
19263              0.000  1.0     0.124    -5.714   0.0   
200896             0.000  1.0     0.118    -3.651   1.0   
215676             0.506  9.0     0.429    -4.087   1.0   

                         name  popularity  speechiness    tempo    genre  
19263      Let Me Down Slowly        84.0       0.0318  150.073  ['pop']  
200896             

In [40]:
#Throw an error to stop the execution here
throw_error()

TypeError: throw_error.__init__() missing 1 required positional argument: 'mess'

In [44]:
from useKeam import FindSimilarSong

f = FindSimilarSong()
result = f.find_songs_for_playlist(track_to_analyse)




### Create Spotify playlist

In [45]:
# Get user ID (required to create a playlist)
user_id = sp.current_user()['id']

# Create a new playlist
playlist_name = "AI-ify Playlist"
playlist_description = "This is a playlist created using AI-ify."
playlist = sp.user_playlist_create(user_id, playlist_name, public=True, description=playlist_description)

# Add tracks to the playlist
sp.playlist_add_items(playlist['id'], result)

{'snapshot_id': 'AAAAAhsf1CmKa1k3QNNpv8xsTDJ84pXt'}